In [1]:
%pip install langchain_community langchain langchain_cohere langchain_groq

  Using cached protobuf-4.25.5-cp310-abi3-win_amd64.whl.metadata (541 bytes)
Using cached protobuf-4.25.5-cp310-abi3-win_amd64.whl (413 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.28.3
    Uninstalling protobuf-5.28.3:
      Successfully uninstalled protobuf-5.28.3
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.28.1 requires protobuf<6.0,>=5.0, but you have protobuf 4.25.5 which is incompatible.


In [2]:
%pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [3]:
import langchain

In [4]:
import os
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain_groq import ChatGroq  


In [5]:
import os 

GROQ_API_KEY='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'
os.environ["GROQ_API_KEY"]='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'

In [6]:
llm= ChatGroq(
            model='llama3-8b-8192',
            temperature=0.6,
            api_key=os.getenv("GROQ_API_KEY")
        )


In [7]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a helpful assistant that tells jokes."),
    HumanMessage(content="Tell me about programming")
]
response = llm.invoke(messages)
print(response)

template = ChatPromptTemplate([
    ("system", "You are a helpful assistant that tells jokes."),
    ("human", "Tell me about {topic}")
])
chain = template | llm
response = chain.invoke({"topic": "programming"})
print(response)


content='Programming! It\'s like trying to explain a joke to a computer. You have to be precise, or it just won\'t get it!\n\nBut seriously, programming is like building with Legos, but instead of blocks, you use code! You start with a foundation, add some logic, and then... voilà! You create something amazing!\n\nHere\'s a joke to illustrate the process:\n\nWhy did the programmer quit his job?\n\nBecause he didn\'t get arrays! (get a raise)\n\nOkay, maybe it\'s a bit of a "bug" (get it?), but I hope it "compiles" a smile on your face!\n\nNow, if you\'re interested in learning more about programming, I\'d be happy to help you "debug" any questions you might have!' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 158, 'prompt_tokens': 28, 'total_tokens': 186, 'completion_time': 0.131666667, 'prompt_time': 0.003600131, 'queue_time': 0.009942199, 'total_time': 0.135266798}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 

**Doccument Processing For RAG**

In [8]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document
import os

def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents

folder_path = r"C:\Users\user\Desktop\Alchemist AI Assignment\data"
documents = load_documents(folder_path)
print(f"Loaded {len(documents)} documents from the folder.")


Loaded 1 documents from the folder.


In [9]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)

splits = text_splitter.split_documents(documents)
print(f"Split the documents into {len(splits)} chunks.")


Split the documents into 115 chunks.


In [11]:
print(splits[1])


page_content='dynasty, ruled over the Indian subcontinent from' metadata={'source': 'C:\\Users\\user\\Desktop\\Alchemist AI Assignment\\data\\The Mughal Empire.pdf', 'page': 0}


Creating Embeddings

In [10]:
embeddings_model_name='embed-english-v2.0'
COHERE_API_KEY='30lAzH7VO1y9M00rSD5ns4gIIxicqxiMnCn1IycB'
import os
os.environ['COHERE_API_KEY']=COHERE_API_KEY 
from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(
        model=embeddings_model_name,
        cohere_api_key=os.getenv('COHERE_API_KEY')
    )

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\user\AppData\Local\sagemaker\sagemaker\config.yaml


In [11]:
embeddings

CohereEmbeddings(client=<cohere.client.Client object at 0x0000020B84C4EF50>, async_client=<cohere.client.AsyncClient object at 0x0000020BE68C4450>, model='embed-english-v2.0', truncate=None, cohere_api_key=SecretStr('**********'), embedding_types=['float'], max_retries=3, request_timeout=None, user_agent='langchain:partner', base_url=None)

In [12]:
import chromadb

In [11]:
import pypika.dialects


In [15]:
from chromadb.config import Settings
from chromadb import Client

# Initialize ChromaDB with persistence
settings = Settings(persist_directory="./chroma_db")
client = Client(settings=settings)

# List available collections
try:
    collections = client.list_collections()
    print(f"Collections: {collections}")
except Exception as e:
    print(f"Error listing collections: {e}")


Collections: []


In [16]:
# Create or get a collection
try:
    collection = client.get_or_create_collection(name="my_collection")
    print("Collection created:", collection)
except Exception as e:
    print(f"Error creating collection: {e}")


Collection created: Collection(name=my_collection)


In [12]:
from langchain.vectorstores import Chroma
collection_name="my_collection"
vectorstore = Chroma.from_documents(
    collection_name=collection_name,
    documents=splits,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

print("Vector store created and persisted to './chroma_db'")


Vector store created and persisted to './chroma_db'


In [17]:
query = "What was babar?"
search_results = vectorstore.similarity_search(query, k=2)
print(f"\nTop 2 most relevant chunks for the query: '{query}'\n")
for i, result in enumerate(search_results, 1):
    print(f"Result {i}:")
    print(f"Source: {result.metadata.get('source', 'Unknown')}")
    print(f"Content: {result.page_content}")
    print()



Top 2 most relevant chunks for the query: 'What was babar?'

Result 1:
Source: C:\Users\user\Desktop\Medical ChatBot\data\The Mughal Empire.pdf
Content: culture with the rich traditions of the Indian subcontinent, resulting in a unique and lasting legacy in art, 
architecture, and administration. 
Babur's grandson, Akbar the Great, is often credited with consolidating and expanding the Mughal 
Empire. Ascending the throne at the tender age of 13, Akbar displayed exceptional military prowess and 
strategic acumen. His policy of religious tolerance and inclusion, known as Sulh-e-Kul or "universal

Result 2:
Source: C:\Users\user\Desktop\Medical ChatBot\data\The Mughal Empire.pdf
Content: culture with the rich traditions of the Indian subcontinent, resulting in a unique and lasting legacy in art, 
architecture, and administration. 
Babur's grandson, Akbar the Great, is often credited with consolidating and expanding the Mughal 
Empire. Ascending the throne at the tender age of 13, Akbar 

In [19]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
# retriever_results = retriever.invoke("Who was Akbar?")
# print(retriever_results)


In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template = """Answer the question based only on the following context:
{context}
Question: {question}
Answer: """

prompt = ChatPromptTemplate.from_template(template)

def docs2str(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [16]:
question = "Who was Akbar?"
response = rag_chain.invoke(question)
print(f"Question: {question}")
print(f"Answer: {response}")


Question: Who was Akbar?
Answer: Akbar was Babur's grandson.


In [17]:
question = "Who was his son?"
response = rag_chain.invoke(question)
print(f"Question: {question}")
print(f"Answer: {response}")

Question: Who was his son?
Answer: Answer: Humayun


In [22]:
question = "What was his religious policy?"
response = rag_chain.invoke(question)
print(f"Question: {question}")
print(f"Answer: {response}")

Question: What was his religious policy?
Answer: The question is asking for information based on the provided context, which only mentions "strategic acumen" and "policy of religious". Therefore, the answer would be "not stated" or "unknown", as there is no specific information provided about his religious policy in the given context.


**History Aware Retriver**

In [25]:
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.output_parsers import StrOutputParser
contextualize_q_system_prompt = """
Given a chat history and the latest user question
which might reference context in the chat history,
formulate a standalone question which can be understood
without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is.
"""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

contextualize_chain = contextualize_q_prompt | llm | StrOutputParser()


In [29]:
print(contextualize_chain.invoke({"input": "Who was Akbar?", "chat_history": []}))


Who was Akbar the Great?


In [32]:
from langchain.chains import create_retrieval_chain

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the following context to answer the user's question.Answer to the point and precise manner."),
    ("system", "Context: {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


**History Aware RAG chain**

In [34]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []
question1 = "Who was Akbar?"
answer1 = rag_chain.invoke({"input": question1, "chat_history": chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question1),
    AIMessage(content=answer1)
])

print(f"Human: {question1}")
print(f"AI: {answer1}\n")

question2 = "Who was his Son?"
answer2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question2),
    AIMessage(content=answer2)
])

print(f"Human: {question2}")
print(f"AI: {answer2}")


Human: Who was Akbar?
AI: Akbar the Great was the third Mughal Emperor of India, ruling from 1556 to 1605. He was the grandson of Babur, the founder of the Mughal Empire.

Human: Who was his Son?
AI: Akbar's son was Jahangir, who succeeded him as the fourth Mughal Emperor of India.


**Combining Every Block of Code**

In [20]:

from langchain_core.documents import Document
import os
from dotenv import load_dotenv
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
from typing import List


# Constants
load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
model_type = os.getenv('MODEL_TYPE')


# Initialize LLM and Embeddings
llm = ChatGroq(
    model=model_type,
    temperature=0.7,
    api_key=os.getenv('GROQ_API_KEY')
)

def create_chains(llm, retriever):
    """Create necessary chains for the RAG pipeline."""
    contextualize_ques_prompt = ChatPromptTemplate.from_messages([
        ("system", """
        Given a chat history and the latest user question
        which might reference context in the chat history,
        formulate a standalone question which can be understood
        without the chat history. Do NOT answer the question,
        just reformulate it if needed and otherwise return it as is.
        """),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ])
    contextualize_chain = contextualize_ques_prompt | llm | StrOutputParser()
    history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_ques_prompt)

    qa_prompt = ChatPromptTemplate.from_messages([
        ("system", """
        You are a helpful RAG AI assistant. 
        Use the following context to answer the user's question. 
        Answer to the point,precise and in a very specific manner.
        If it is an out-of-context question, do not answer and say it is out of context.
        """),
        ("system", "Context: {context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}")
    ])
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
    return rag_chain


In [21]:
from langchain.chains import LLMChain, MultiRouteChain
from langchain.chains.router import RouterChain
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

def create_generic_prompt_chain(llm):
    """Create a chain to handle vague/generic inputs."""
    generic_prompt = ChatPromptTemplate.from_messages([
        ("system", """
        The user's input seems vague or generic. Your task is to guide the user
        to provide a more specific input. Do NOT answer the query; instead, 
        politely ask for clarification or specificity.
        """),
        ("human", "{input}")
    ])
    generic_chain = LLMChain(llm=llm, prompt=generic_prompt)
    return generic_chain

def create_router_chain(llm, retriever):
    """Combine RAG chain and generic handling chain into a RouterChain."""
    # Create the history-aware RAG chain
    rag_chain = create_chains(llm, retriever)

    # Create the generic prompt chain
    generic_chain = create_generic_prompt_chain(llm)

    # Define the router logic
    router_prompt = PromptTemplate(
        input_variables=["input"],
        template="""
        Decide whether the following input is 'specific' or 'generic':
        Input: {input}
        Respond with 'specific' or 'generic' only.
        """
    )
    router_chain = LLMChain(llm=llm, prompt=router_prompt)

    # Combine chains with RouterChain
    router = RouterChain(
        router_chain=router_chain,
        destination_chains={
            "specific": rag_chain,
            "generic": generic_chain,
        },
        default_chain=generic_chain,  # Fallback to generic if classification fails
    )
    return router


router_chain = create_router_chain(llm, retriever)

# Test input
user_input = "Tell me something interesting."
response = router_chain.run({"input": user_input})
print(response)


C:\Users\user\AppData\Local\Temp\ipykernel_14884\1589971558.py:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  generic_chain = LLMChain(llm=llm, prompt=generic_prompt)


TypeError: Can't instantiate abstract class RouterChain with abstract methods _call, input_keys

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chat_models import ChatOpenAI

def create_generic_prompt_chain(llm):
    """Create a chain to handle vague/generic inputs."""
    generic_prompt = ChatPromptTemplate.from_messages([
        ("system", """
        The user's input seems vague or generic. Your task is to guide the user
        to provide a more specific input. Do NOT answer the query; instead, 
        politely ask for clarification or specificity.
        """),
        ("human", "{input}")
    ])
    generic_chain = LLMChain(llm=llm, prompt=generic_prompt)
    return generic_chain

def router_logic(llm, user_input):
    """Decides if the input is 'specific' or 'generic'."""
    router_prompt = PromptTemplate(
        input_variables=["input"],
        template="""
        Based on the input provided, classify it as either 'specific' or 'generic':
        Input: {input}
        Respond with either 'specific' or 'generic' only.
        """
    )
    router_chain = LLMChain(llm=llm, prompt=router_prompt)
    classification = router_chain.run({"input": user_input})
    return classification.strip().lower()

def create_router_chain(llm, retriever):
    """Combines RAG and generic chains with routing logic."""
    # Create the history-aware RAG chain
    rag_chain = create_chains(llm, retriever)

    # Create the generic chain
    generic_chain = create_generic_prompt_chain(llm)

    # Routing function
    def route_input(user_input):
        classification = router_logic(llm, user_input)
        if classification == "specific":
            return rag_chain.run({"input": user_input})
        elif classification == "generic":
            return generic_chain.run({"input": user_input})
        else:
            # Fallback for unclassified cases
            return "I'm sorry, I couldn't classify your input. Please try again."

    return route_input


router_chain = create_router_chain(llm, retriever)




In [29]:

user_input = "Tell me something interesting."
response = router_chain(user_input)
response

"I'd love to share something interesting with you! However, I'd like to clarify what kind of interesting you're looking for.\n\nAre you interested in learning something new about a particular topic, such as science, history, or technology? Or perhaps you're looking for something more lighthearted, like a fun fact or a surprising statistic?\n\nCould you please give me a bit more context or guidance on what you'd find interesting? That way, I can provide a more tailored response that meets your curiosity!"

In [31]:
def create_router_chain(llm, retriever,user_input):
    """Combines RAG and generic chains with routing logic."""
    # Create the history-aware RAG chain
    rag_chain = create_chains(llm, retriever)

    # Create the generic chain
    generic_chain = create_generic_prompt_chain(llm)

    # Routing function
    def route_input(user_input):
        classification = router_logic(llm, user_input)
        if classification == "specific":
            return rag_chain.invoke({
                "input": user_input
                
            })['answer']
            
        elif classification == "generic":
            return generic_chain.run({"input": user_input})
        else:
            # Fallback for unclassified cases
            return "I'm sorry, I couldn't classify your input. Please try again."

    return route_input


In [32]:

user_input = "Tell me something interesting."
response = create_router_chain(llm, retriever,user_input)
response

<function __main__.create_router_chain.<locals>.route_input(user_input)>